# Pavyzdinis viešbučių ir skrydžių užsakymo agentas

Ši sistema padės užsakyti skrydžio bilietus ir viešbutį. Scenarijus: kelionė iš Londono Heathrow LHR 2024 m. vasario 20 d. į Niujorką JFK su grįžimu 2025 m. vasario 27 d., skrydžiai ekonomine klase tik su British Airways. Noriu apsistoti Hilton viešbutyje Niujorke, prašau pateikti skrydžio ir viešbučio kainas.


# Inicializuokite Azure AI Agent Service ir gaukite konfigūracijos informaciją iš **.env**

### **.env**

Sukurkite .env failą

**.env** failas turi Azure AI Agent Service jungties eilutę, AOAI naudojamą modelį ir atitinkamus Google API Search paslaugos API, ENDPOINT ir kt.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Jūsų Azure AI Agent Service modelio dislokacijos pavadinimas"

[**PASTABA**] Jums reikės modelio su 100 000 greičio limitu (žetonai per minutę) ir 600 greičio limitu (užklausų per minutę)

 Jūs galite gauti modelį Microsoft Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Jūsų Azure AI Agent Service projekto jungties eilutė"

  Projekto jungties eilutę galite rasti savo projekto apžvalgoje AI Foundry portalo ekrane.

- **SERPAPI_SEARCH_API_KEY** = "Jūsų SERPAPI Search API RAKTAS"
- **SERPAPI_SEARCH_ENDPOINT** = "Jūsų SERPAPI Search galinė taškas"

Norėdami gauti Azure AI Agent Service modelio dislokacijos pavadinimą ir projekto jungties eilutę, turite sukurti Azure AI Agent Service. Rekomenduojama naudoti [šį šabloną](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json), kad sukurtumėte jį tiesiogiai (***Pastaba:*** Azure AI Agent Service šiuo metu yra nustatytas ribotoje regione. Rekomenduojama pasitikrinti [šią nuorodą](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) regiono nustatymui)

Agentas turi turėti prieigą prie SERPAPI. Rekomenduojama užsiregistruoti naudojant [šią nuorodą](https://serpapi.com/searches). Po registracijos galite gauti unikalų API RAKTĄ ir GALINĮ TAŠKĄ


# Setup 

Norėdami paleisti šį užrašų knygelės failą, turite įsitikinti, kad įdiegėte reikiamas bibliotekas, paleisdami `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Jūsų Semantic Kernel versija turėtų būti bent 1.27.2.


Įkelkite savo .env failo nustatymus ir išteklius, prašome įsitikinti, kad pridėjote savo raktus ir nustatymus bei sukūrėte vietinį .env failą.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Prisijunkite prie Azure

Dabar turite prisijungti prie Azure. Atidarykite terminalą ir vykdykite šią komandą:

```bash
az login
```

Ši komanda paprašys įvesti jūsų Azure prisijungimo duomenis, kad Azure AI Agent paslauga veiktų tinkamai.


# Paaiškinimas:
Tai kintamasis, kuris saugo API raktą, skirtą naudoti SERP (Paieškos variklio rezultatų puslapio) API paslaugai. API raktas yra unikalus identifikatorius, naudojamas autentiškuoti užklausas, susietas su jūsų paskyra.

Paskirtis: Šios eilutės paskirtis yra saugoti API raktą kintamajame, kad jis galėtų būti naudojamas autentiškuoti užklausas SERP API paslaugai. API raktas reikalingas, kad būtų galima pasiekti paslaugą ir atlikti paieškas.
Kaip gauti SERP API raktą: Norėdami gauti SERP API raktą, atlikite šiuos bendruosius veiksmus tinklalapyje https://serpapi.com (tiksli procedūra gali skirtis priklausomai nuo konkrečios naudojamos SERP API paslaugos):

Pasirinkite SERP API paslaugą: Yra keletas SERP API paslaugų, tokių kaip SerpAPI, Google Custom Search JSON API ir kitos. Pasirinkite tą, kuri geriausiai atitinka jūsų poreikius.

Užsiregistruokite paskyrai: Eikite į pasirinktos SERP API paslaugos svetainę ir užsiregistruokite paskyrai. Gali tekti pateikti pagrindinę informaciją ir patvirtinti savo el. pašto adresą.

Sukurkite API raktą: Užsiregistravę, prisijunkite prie paskyros ir eikite į API skyrių arba prietaisų skydelį. Ieškokite galimybės sukurti arba generuoti naują API raktą.
Kopijuokite API raktą į savo .env failą.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Paaiškinimas:
BASE_URL: Tai yra kintamasis, kuris saugo pagrindinį URL SERP API pabaigos taškui. Kintamojo pavadinimas BASE_URL yra konvencija, naudojama nurodyti, kad šis URL yra pradinė vieta API užklausoms atlikti.
'https://serpapi.com/search':

Tai yra faktinis URL eilutė, priskirta kintamajam BASE_URL. Jis žymi pabaigos tašką, skirtą ieškos užklausoms atlikti naudojant SERP API.

# Tikslas:
Šios eilutės tikslas – apibrėžti konstantą, kuri laiko pagrindinį URL SERP API. Šis URL bus naudojamas kaip pradinis taškas konstrukcijai API užklausoms atlikti ieškos operacijoms.

# Naudojimas:
Apibrėždami pagrindinį URL kintamajame, galite jį lengvai pakartotinai naudoti visame savo kode, kai tik reikia atlikti užklausas SERP API. Tai daro jūsų kodą lengviau prižiūrimą ir sumažina klaidų riziką, kai URL yra įrašytas keliose vietose. Dabartinis pavyzdys yra https://serpapi.com/search?engine=bing, kuris naudoja Bing paieškos API. Skirtingą API galima pasirinkti <https://Serpapi.com>


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Paaiškinimas:

Čia yra jūsų įskiepio kodas.

Klasės apibrėžimas: `class BookingPlugin`: Apibrėžia klasę pavadinimu BookingPlugin, kuri turi metodus skrydžių ir viešbučių užsakymui.

Viešbučio užsakymo metodas:

- `@kernel_function(description="booking hotel")`: Dekoratorius, kuris aprašo funkciją kaip branduolio funkciją viešbučių užsakymui.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Apibrėžia metodą viešbučių užsakymui su anotuotais parametrais ir grąžinimo tipu.

Metodas sukuria žodyną su parametrais viešbučio užsakymo užklausai ir siunčia GET užklausą į SERP API. Jis patikrina atsakymo būseną ir jei sėkminga, gražina viešbučio ypatybes, arba None, jei užklausa nepavyko.

Skrydžio užsakymo metodas:

- `@kernel_function(description="booking flight")`: Dekoratorius, kuris aprašo funkciją kaip branduolio funkciją skrydžių užsakymui.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Apibrėžia metodą skrydžių užsakymui su anotuotais parametrais ir grąžinimo tipu.

Metodas sukuria žodynus su parametrais išvykimo ir grįžimo skrydžių užklausoms ir siunčia GET užklausas į SERP API. Jis patikrina atsakymo būseną ir jei sėkminga, prideda skrydžių informaciją prie rezultatų eilutės, arba išspausdina klaidos pranešimą, jei užklausa nepavyko. Metodas gražina rezultatų eilutę, kurioje yra skrydžių informacija.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Paaiškinimas:  
Importo sakiniai: Importuojami būtini moduliai Azure kredencialams, AI agentui, pokalbio žinutės turiniui, autoriaus vaidmeniui ir kerneliui žymėti.  

Asinchroninis kontekstų valdiklis: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Nustatomas asinchroninis konteksto valdiklis Azure kredencialams tvarkyti ir AI agento klientui kurti.  

Agentų pavadinimas ir instrukcijos:  
- `AGENT_NAME = "BookingAgent"`: Nustato agento pavadinimą.  
- `AGENT_INSTRUCTIONS = """..."""`: Pateikia detalias instrukcijas agentui, kaip tvarkyti užsakymų užklausas.  

Agentų apibrėžimo kūrimas: `agent_definition = await client.agents.create_agent(...)`: Sukuria agento apibrėžimą su nurodytu modeliu, pavadinimu ir instrukcijomis.  

AzureAI agente kūrimas: `agent = AzureAIAgent(...)`: Sukuria AzureAI agentą naudojant klientą, agento apibrėžimą ir apibrėžtą papildinį.  

Sriegio kūrimas: `thread: AzureAIAgentThread | None = None`: Sukuria agento sriegių. Pirmiausia sriegių kūrimas nėra privalomas - jei nurodyta reikšmė `None`, naujas sriegis bus sukurtas pirmojo kvietimo metu ir grąžintas atsakyme.  

Vartotojo įvestys: `user_inputs = ["..."]`: Nustato vartotojo įvedimų sąrašą, kurį agentas turi apdoroti.  

Pagal finally bloką ištrinkite sriegius ir agentą, kad būtų išvalyti resursai.


# Autentifikacija

Klasė `DefaultAzureCredential` yra Azure SDK dalis Python kalbai. Ji suteikia numatytą būdą autentifikuotis Azure paslaugoms. Ji bando autentifikuotis naudodama kelis metodus tam tikra tvarka, pavyzdžiui, aplinkos kintamuosius, valdomą identitetą ir Azure CLI kredencialus.

Asinchroninės operacijos: aio modulis rodo, kad DefaultAzureCredential klasė palaiko asinchronines operacijas. Tai reiškia, kad galite ją naudoti su asyncio, norint atlikti neblokuojančius autentifikacijos užklausimus.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Atsakomybės apribojimas**:
Šis dokumentas buvo išverstas naudojantis dirbtinio intelekto vertimo paslauga [Co-op Translator](https://github.com/Azure/co-op-translator). Nors stengiamės užtikrinti tikslumą, prašome atkreipti dėmesį, kad automatizuoti vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Svarbiai informacijai rekomenduojamas profesionalus vertimas žmogaus. Mes neatsakome už jokius nesusipratimus ar neteisingus aiškinimus, kilusius dėl šio vertimo naudojimo.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
